In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
tf.random.set_seed(3)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from collections import Counter

In [2]:
tf.__version__

'2.11.0'

In [ ]:
import splitfolders

data_dir = r"D:\Research Internship\DrowsinnessDetection\DD"
output_dir = r"D:\Research Internship\DrowsinnessDetection\splitted_Data"
splitfolders.ratio(data_dir, output=output_dir, seed=1337, ratio=(.8, 0.15, 0.05))

Copying files: 1989 files [01:25, 32.44 files/s]

In [ ]:
train_dir = r"D:\Research Internship\DrowsinnessDetection\splitted_Data\train"
test_dir = r"D:\Research Internship\DrowsinnessDetection\splitted_Data\test"
val_dir = r"D:\Research Internship\DrowsinnessDetection\splitted_Data\val"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_batches = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    shuffle=True
)

In [ ]:
test_batches = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary' ,
    shuffle=True
)

In [ ]:
val_batches = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    shuffle=True
)

In [ ]:
train_class_indices = train_batches.class_indices
test_class_indices = test_batches.class_indices
val_class_indices = val_batches.class_indices

train_class_labels = train_batches.classes
test_class_labels = test_batches.classes
val_class_labels = val_batches.classes


train_class_counts = Counter(train_class_labels)
test_class_counts = Counter(test_class_labels)
val_class_counts = Counter(val_class_labels)

print("Class Names for train:\n", train_class_indices)
print("Class Counts for train:\n", train_class_counts)
print(end='\n')

print("Class Names for test:\n", test_class_indices)
print("Class Counts for test:\n", test_class_counts)
print(end='\n')

print("Class Names for validation :\n", val_class_indices)
print("Class Counts for validation:\n", val_class_counts)

In [ ]:
images, labels = next(train_batches)
print(f"Pixels of the first image after Normalization: \n\n{images[0]}") #print pixels of the first img
plt.imshow(images[0])
plt.show()

In [ ]:
print(f"there are { images[0].ndim} Channels ")
print(f"image shape : {images[0].shape}")

In [ ]:
fig, axes = plt.subplots(8, 4, figsize=(15, 30))
class_indices = train_batches.class_indices

for i in range(8):
    images, labels = next(train_batches)
    for j in range(4):

        ax = axes[i, j]
        ax.imshow(images[j])
        ax.axis('off')
        label = int(labels[j])
        label_name = list(class_indices.keys())[list(class_indices.values()).index(label)]
        ax.set_title(f'{label_name} ({label})')

plt.tight_layout()
plt.show()

In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
)

In [ ]:
type(base_model)

In [ ]:
base_model.summary()

In [ ]:
model=keras.Sequential()

In [ ]:
for layer in base_model.layers[:-25] :
    layer.trainable = False

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(0.0001 ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_batches,
    epochs=10,
    validation_data=val_batches,
    batch_size=32
)

Epoch 1/10
2090/2090 [==============================] - 2610s 1s/step - loss: 0.0292 - accuracy: 0.9938 - val_loss: 0.0081 - val_accuracy: 0.9984
Epoch 2/10
2090/2090 [==============================] - 2334s 1s/step - loss: 0.0086 - accuracy: 0.9989 - val_loss: 0.0424 - val_accuracy: 0.9877
Epoch 3/10
2090/2090 [==============================] - 2311s 1s/step - loss: 0.0130 - accuracy: 0.9980 - val_loss: 0.0019 - val_accuracy: 0.9997
Epoch 4/10
2090/2090 [==============================] - 2395s 1s/step - loss: 0.0106 - accuracy: 0.9987 - val_loss: 0.0045 - val_accuracy: 0.9994
Epoch 5/10
2090/2090 [==============================] - 19619s 9s/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.0014 - val_accuracy: 0.9998
Epoch 6/10
2090/2090 [==============================] - 2349s 1s/step - loss: 0.0043 - accuracy: 0.9994 - val_loss: 0.0438 - val_accuracy: 0.9974
Epoch 7/10
2090/2090 [==============================] - 2430s 1s/step - loss: 0.0069 - accuracy: 0.9993 - val_loss: 0.0033 

In [25]:
# model.save("D:\Machine Learning\Projects\ResearchInternship\DDD.h5")

In [ ]:
model = load_model("D:\Machine Learning\Projects\ResearchInternship\DDD.h5")

In [ ]:
pd.DataFrame(history.history).plot(figsize = (12,10))
plt.xlabel("Epoch")
plt.ylabel("Accuracy and loss")
plt.show()

In [ ]:
pd.DataFrame(history.history)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
loss, accuracy = model.evaluate(train_batches)
print(f"Training Loss: {loss:.4f}")
print(f"Training Accuracy: {accuracy*100:.2f}%")

In [ ]:
loss, accuracy = model.evaluate(test_batches)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:


y_pred = model.predict(test_batches)
y_pred

In [ ]:
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels

In [ ]:
y_actual = test_batches.labels
y_actual

In [ ]:
conf_matrix = confusion_matrix(y_actual, y_pred_labels)

print(conf_matrix)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='bone',
            xticklabels=['Class 0', 'Class 1'],
            yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:


report = classification_report(y_actual, y_pred_labels)
print(report)



In [ ]:
import visualkeras
from keras.models import  load_model
model = load_model("D:\Machine Learning\Projects\ResearchInternship\DDD.h5")

In [ ]:
visualkeras.layered_view(model).show()
visualkeras.layered_view(model,legend=True)